* Steps for quantitative analysis 
* run dump circuits to create a QS version of each mutant 
* Fix the QS mutants if there are any errors / namespace, filename and test file name/ syntax errors
* Make a test file for the property
* run the fg file to build and make the runner
* run on this pc

In [ ]:
from qiskit import qasm2
import subprocess
import importlib.util
import sys
import os

PATH = os.path.abspath("")

subprocess.run(["npm -g install q-convert"], shell=True)

def import_function(module_str, path, function_name):
    spec = importlib.util.spec_from_file_location(module_str, path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_str] = module
    spec.loader.exec_module(module)
    return getattr(module, function_name)


In [ ]:
def process_mutant(algorithm_name, mutant_suffix, index, *args):
    mutant_name = f"{algorithm_name}_{mutant_suffix}{index}"
    circuit_function = import_function(mutant_name, f"{PATH}/{algorithm_name}/mutants/{mutant_name}.py", algorithm_name)
    circuit = circuit_function(*args) if args else circuit_function()
        
    filename = f"{PATH}/{algorithm_name}/mutants/translated_circuits/{mutant_name}.qasm"
    
    dir_path = os.path.dirname(filename)
    if dir_path and not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    qasm2.dump(circuit, filename)
    
    command = [
        "q-convert", "-i", filename,
        "-s", "qasm", "-o", f"{PATH}\\{algorithm_name}\\mutants\\translated_circuits\\{mutant_name}.qs",
        "-d", "qsharp", "-w"
    ]
    
    process = subprocess.run(command, shell=True, capture_output=True, text=True)
    
def dump_circuits(algorithm_name, *args):
    for i in range(10):
        process_mutant(algorithm_name, "m", i, *args)
    
    for i in range(5):
        process_mutant(algorithm_name, "em", i, *args)

In [ ]:
dump_circuits("quantum_teleportation")

In [ ]:
dump_circuits("quantum_fourier_transform", 5)